Tri Cities Model

In [ ]:
import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

from scipy import linalg
import pandas as pd
import networkx as nx
from numpy import random
import inspect
import sys
import weakref
from EpiCommute import SIRModel
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
np.set_printoptions(precision = 5, suppress = True, linewidth = 100)
from IPython.display import clear_output, display
from ipywidgets import interactive, interact

# Declare Objects

In [ ]:
#Highway is a class that holds the weight of the pathway between cities and all of the connections formed by the highway
class Highway:
    instances=[]
    def __init__(self,name,weight=0,cities=np.array([[]])):
        self.__class__.instances.append(self)
        self.name=name
        self.weight=weight
        self.cities=cities
    


In [ ]:
#City simply holds the population and location of each city
class City:
    instances=[]
    def __init__(self,name,pop=0,loc=[0,0]):
        self.__class__.instances.append(self)
        self.name=name
        self.pop=pop
        self.loc=loc
        

In [ ]:
#Declaring Cities(populations and locations are accurate)
Johnson_City = City('Johnson_City',66515,[36.3345,-82.3703])
Bristol =City('Bristol',26852,[36.5951,-82.1887])
Jonesborough = City('Jonesborough',5247,[36.2943,-82.4735])
Elizabethton = City('Elizabethton',13577,[36.3487,-82.2107])
Gray= City('Gray',1016,[36.4172,-82.4720])
Kingsport = City('Kingsport',53376,[36.5484,-82.5618])
#Declaring Highways
I_26 = Highway('I_26',63665,np.array([['Kingsport','Gray'],['Gray','Johnson_City']]))
I_81 = Highway('I_81',40277,np.array([['Gray','Bristol']]))
Three_21=Highway('Three_21',26322,np.array([['Jonesborough','Johnson_City'],['Johnson_City','Elizabethton']]))
Nineteen_e = Highway('Nineteen_e',9370,np.array([['Elizabethton','Bristol']]))
Eleven_w = Highway('Eleven_w',16644,np.array([['Kingsport','Bristol']]))
Seventy_Five=Highway('Seventy_Five',5762,np.array([['Gray','Jonesborough']]))

In [ ]:
Pre_Highway_List=[]
for instance in Highway.instances:
    Pre_Highway_List.append(instance.name)
#Remove duplicates
    Highway_List=[]
for i in Pre_Highway_List:
    if i not in Highway_List:
        Highway_List.append(i)
Highway_List

# Model setup

## Create Graph




In [ ]:
from numpy import random
#Create Tri_Cities Graph
Tri_Cities = nx.Graph()
#Nodes are connected with self via random weights


In [ ]:
# Populate the graph with weighted edges using the list of highways
for a in range(len(Highway_List)):
    HW=globals()[Highway_List[a]]
    for b in range(len(HW.cities)):
        Tri_Cities.add_edge(HW.cities[b][0],HW.cities[b][1],weight=HW.weight)
list(Tri_Cities.nodes)

In [ ]:
pop_array=np.zeros(len(Tri_Cities.nodes))#pop_array lists the populations of all cities
for a in range(len(Tri_Cities.nodes)):
    pop_array[a]=eval(list(Tri_Cities.nodes)[a]).pop
pop_array  

## Mobility

In [ ]:
norm_pop = pop_array/sum(pop_array)
mobility_1=nx.adj_matrix(Tri_Cities).toarray()#create weighted transition matrix
mobility_t=np.zeros((6,6))
for a in range(6):
    mobility_t[a,:]=mobility_1[a]/sum(mobility_1[a])
mobility_t
difff= 3# diff will represent the output of the cost function later
for z in range(500):#z will represent the scaler multiple of the population array for SAH weights
    rand=pop_array*.1*z/max(pop_array)
    mobility_2 =mobility_t+np.diag(rand)
    mobility_2
    for a in range(6):
        mobility_2[a,:]=mobility_2[a,:]/sum(mobility_2[a,:])#normalize transition matrix
    eigs=np.linalg.eig(mobility_2.T)
    whereat=np.where(abs(eigs[0]-1)<.01)[0][0]# find dominant eigenvector
    stab_dist=eigs[1][:,whereat]/sum(eigs[1][:,whereat])
    if(sum((stab_dist-norm_pop)*(stab_dist-norm_pop))<difff):# use ols to find the lowest cost function
        answer=rand
        difff=sum((stab_dist-norm_pop)*(stab_dist-norm_pop))
    
mobility_2=mobility_t +np.diag(answer)
for a in range(6):
    mobility_2[a,:]=mobility_2[a,:]/sum(mobility_2[a,:])
mobility_2# normalized final transition matrix


## Run Model

In [ ]:
# outbreak source is city of our choice

examples={}# examples will save variations

City_List=list(Tri_Cities.nodes())
# Initialize the model
variations=100
for a in range(6):
    for b in range(variations):
        model = SIRModel(
            mobility_2,#transition matrix
            pop_array,#susbpopulations sizes
            outbreak_source=a,
            mu=1/8,# recovery rate
            R0= 2.5,        
            dt=0.1,               # time interval 
            dt_save=1/6,            # when to save observables
            I0=400,# initial infected
            VERBOSE=True
               
                                
            )
        examples['result'+str(b)+list(Tri_Cities.nodes())[a]]=model.run_simulation()
  #save results of simulations 


## Results

In [ ]:
names=list(examples)
examples
#names saves the names of all elements in "examples". This allows the dictionary to be more easily accessed.

In [ ]:
results={}
results['I']=0
results['R']=0
results['S']=0
for a in range(6*variations):
        results['I']+=(1/(6*variations))*np.array(examples[names[a]]['I'])
        results['R']+=(1/(6*variations))*np.array(examples[names[a]]['R'])
        results['S']+=(1/(6*variations))*np.array(examples[names[a]]['S'])
for a in range(6):
    results[City_List[a]+'I_total']=0
    results[City_List[a]+'R_total']=0
    results[City_List[a]+'S_total']=0
    for b in range(variations):
        results[City_List[a]+'I_total']+=(1/variations)*np.array(examples[names[a*variations +b]]['I_total'])
        results[City_List[a]+'R_total']+=(1/variations)*np.array(examples[names[a*variations +b]]['R_total'])
        results[City_List[a]+'S_total']+=(1/variations)*np.array(examples[names[a*variations +b]]['S_total'])
print(results['I'])
        #results is an average of all variations

In [ ]:
if len(np.where(results['Johnson_CityI_total']>.25)[0]>0):
    beginning=np.where(results['Johnson_CityI_total']>.25)[0][0]
    end=np.where(results['Johnson_CityI_total']>.25)[0][-1]        
    amount=results['Johnson_CityR_total'][end]-results['Johnson_CityR_total'][beginning]
else:
    amount=0
amount
# This section calculates the percentage of people infected that are above the estimated hospital capacity of 40%.

In [ ]:
cities={}# cities saves the products of research based on which city is the outbreak source
stds={}# will be used to save standard deviation
means={}# will be used to save means
for a in range(6):# iterated over 6 because there are 6 cities
    cities[City_List[a]+"endvec"]=np.array([])
    cities[City_List[a]+"maxvec"]=np.array([])
    cities[City_List[a]+"infectedvec"]=np.array([])
    for b in range(variations):
        current=examples['result'+str(b)+City_List[a]]
        c_S=np.array(current['S']).T[a]#saves susceptible, infected, recovered for each data set
        c_I=np.array(current['I']).T[a]
        c_R=np.array(current['R']).T[a]
        mx=max(c_I)*sum(pop_array)#maximum infected at once
        I_t = c_R[len(c_R)-1]*sum(pop_array)#total infected
        Half_life= np.where(np.cumsum(c_I)<.5*np.cumsum(c_I))[len(c_I)-1][0]# half the time required for the epidemic to occur
        Half_life=Half_life[len(Half_life)-1]
        cities[City_List[a]+"endvec"]=np.append(cities[City_List[a]+"endvec"],2*Half_life)# saves the end points for the epidemic
        cities[City_List[a]+"infectedvec"]=np.append(cities[City_List[a]+"infectedvec"],I_t)#saves the total infeced
        cities[City_List[a]+"maxvec"]=np.append(cities[City_List[a]+"maxvec"],mx)#saves maximum infected at one time
    stds[str(City_List[a])+'endvecstd']=np.std(cities[City_List[a]+"endvec"])#stanard deviation and mean are found for all
    means[str(City_List[a])+'endvecmean']=np.mean(cities[City_List[a]+"endvec"])
    stds[str(City_List[a])+'maxvecstd']=np.std(cities[City_List[a]+"maxvec"])
    means[str(City_List[a])+'maxvecmean']=np.mean(cities[City_List[a]+"maxvec"])
    stds[str(City_List[a])+'infectedvecstd']=np.std(cities[City_List[a]+"infectedvec"])
    means[str(City_List[a])+'infectedvecmean']=np.mean(cities[City_List[a]+"infectedvec"])
#The following is identical to above, but considers all variations equally
totendvec=np.array([])
totmaxvec=np.array([])
totinfectedvec=np.array([])
for b in range(6*variations):
    current=examples[names[b]]
    c_S=current['S_total']
    c_I=current['I_total']
    c_R=current['R_total']
    mx=max(c_I)*sum(pop_array)
    I_t = c_R[len(c_R)-1]*sum(pop_array)
    Half_life= np.where(np.cumsum(c_I)<.5*np.cumsum(c_I))[len(c_I)-1][0]
    Half_life=Half_life[len(Half_life)-1]
    totendvec=np.append(totendvec,2*Half_life)
    totinfectedvec=np.append(totinfectedvec,I_t)
    totmaxvec=np.append(totmaxvec,mx)
stds['totendvecstd']=np.std(totendvec)
means['totendvecmean']=np.mean(totendvec)
stds['totmaxvecstd']=np.std(totmaxvec)
means['totmaxvecmean']=np.mean(totmaxvec)
stds['totinfectedvecstd']=np.std(totinfectedvec)
means['totinfectedvecmean']=np.mean(totinfectedvec)
means
totinfectedvec


In [ ]:
names_2 = ['endvec','maxvec','infectedvec']
intervals2={}
# this section creates 95% confidence intervals for set of variations
# this takes the mean and adds/ subtracts 1.96* std
for a in range(3):
    intervals2['tot'+names_2[a] + 'interval']= [means['tot'+names_2[a]+'mean']-1.96*stds['tot'+names_2[a]+'std'],means['tot'+names_2[a]+'mean']+1.96*stds['tot'+names_2[a]+'std']]
 
for a in range(3):
    for b in range(6):
        upper=means[list(Tri_Cities.nodes())[b]+names_2[a]+'mean']+1.96*stds[list(Tri_Cities.nodes())[b]+names_2[a]+'std']
        lower=means[list(Tri_Cities.nodes())[b]+names_2[a]+'mean']-1.96*stds[list(Tri_Cities.nodes())[b]+names_2[a]+'std']
        intervals2[list(Tri_Cities.nodes())[b]+names_2[a] + 'interval']=[lower,upper] 

print(intervals2)

cities_list=list(cities)

In [ ]:
fig,ax=plt.subplots(3,figsize=(6.5,10))
ax[0].hist(totendvec)
ax[0].set_xlabel('Epidemic Length in Days')
ax[0].axvline(intervals2['totendvecinterval'][0])
ax[0].axvline(intervals2['totendvecinterval'][1])
ax[1].hist(totmaxvec)
ax[1].set_xlabel('Maximum Infected')
ax[1].axvline(intervals2['totmaxvecinterval'][0])
ax[1].axvline(intervals2['totmaxvecinterval'][1])
ax[2].hist(totinfectedvec)
ax[2].set_xlabel('Total Infected')
ax[2].axvline(intervals2['totinfectedvecinterval'][0])
ax[2].axvline(intervals2['totinfectedvecinterval'][1])

## Epidemic curve

In [ ]:
#plot all compartments as a function of time
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
matplotlib.rc('figure', dpi=200)
#plot SIR.
t=examples[names[0]]['t']
figure = plt.figure(figsize=(5.5,4))
plt.plot(t, results['Johnson_CityS_total'],label='S', color='purple')
plt.plot(t, results['Johnson_CityI_total'], label='I', color='firebrick')
plt.plot(t, results['Johnson_CityR_total'], label='R', color='k')
plt.axhline(.25)# represents the cutoff point for the hospital system
plt.legend(frameon=False, loc='center right')
plt.xlabel("Time")
plt.ylabel("Compartment")
plt.show()

## Spread of infected

In [ ]:
from matplotlib.animation import FuncAnimation
from matplotlib import rc
rc('animation', html='html5')
%config InlineBackend.figure_format = 'svg'

In [ ]:
# this creates dictionaries for the graph plotting function to access
lbls={}
for a in range(6):
    lbls[list(Tri_Cities.nodes())[a]]=list(Tri_Cities.nodes())[a]
poss={}
for a in range(6):
    poss[City_List[a]]=np.flip(eval(City_List[a]).loc)
poss    
results['Johnson_CityI_total']

In [ ]:
#create animation
fig = plt.Figure(figsize = (3,3))
ax = fig.add_subplot()

def f(t=0):
    t=2*t
    ax.clear()
    ax.set_title(t)
    ax.set_ylim([36.2,36.7])
    ax.set_xlim([-82.65,-82.1])
    nx.draw_networkx(Tri_Cities,poss,
               with_labels = True,
               labels = lbls,
               font_size=8,
               node_size = 50, 
               edge_color = 'gray',
               node_color = results['I'][t],
               cmap = 'Reds', ax  = ax,
               vmin = 0,  vmax = 1)
    


anim =FuncAnimation(fig, f, frames=50, interval=100, 
                     blit=False, repeat=True)
anim
